In [1]:
import networkx as nx
import dwave_networkx as dnx
import matplotlib.pyplot as plt
import random
import dimod
from dwave.system import DWaveSampler, EmbeddingComposite, FixedEmbeddingComposite
import dwave.inspector
from pyqubo import Spin
import neal
import time
import numpy as np
import pandas as pd
from greedy import SteepestDescentSolver

### Write SAT_instance.wcnf according to D'Wave architecture

In [62]:
def generate_sat(a,b):
    couplers_indices = DWaveSampler().edgelist
    qubit_indices = DWaveSampler().nodelist

    num_var=0
    num_clauses=0
    qubit=[]
    SAT=""
    for coupler in couplers_indices:
        if a <= coupler[0] <= b:
            for c in coupler:
                if c not in qubit:
                    num_var += 1
                    qubit.append(c)
            num_clauses += 1
            SAT += ("1 " + str(random.choice([coupler[0], -coupler[0]])) + " " + str(random.choice([coupler[1], -coupler[1]])) + " 0\n")
    return SAT, num_var, num_clauses

In [63]:
SAT, num_var, num_clauses=generate_sat(300,310)

In [78]:
print("Num_vars: ", num_var, " ; Num_clauses: ", num_clauses)

Num_vars:  155  ; Num_clauses:  154


In [64]:
file_name = "SAT_instance.wcnf"
#header="c WCNF exact solver\np wcnf "+str(num_var)+" "+str(num_clauses)+"\n"
with open(file_name, 'w') as archivo:
    #archivo.write(header)
    archivo.write(SAT)

### Generate ising hamiltonian from a SAT file (WCNF)

In [65]:
def parsear_cnf(file_name):
    clauses=[]
    with open(file_name, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if not line.startswith('c') and not line.startswith('p'):
                clauses.append(list(map(int, line.split()[1:-1])))
    return clauses

In [66]:
clauses = parsear_cnf(file_name)

In [67]:
h={}
J={}
qubits=[]
couple=[]
for clause in clauses:
    if np.abs(clause[0]) not in qubits:
        h[np.abs(clause[0])] = (0.25 if clause[0]<0 else -0.25)
        qubits.append(np.abs(clause[0]))
    else:
        h[np.abs(clause[0])] += (0.25 if clause[0]<0 else -0.25)
    if np.abs(clause[1]) not in qubits:
        h[np.abs(clause[1])] = (0.25 if clause[1]<0 else -0.25)
        qubits.append(np.abs(clause[1]))
    else:
        h[np.abs(clause[1])] += (0.25 if clause[1]<0 else -0.25)
    if [np.abs(clause[0]),np.abs(clause[1])] not in couple:
        J[np.abs(clause[0]),np.abs(clause[1])] = (0.25 if clause[0]*clause[1]>0 else -0.25)
        couple.append([clause[0],clause[1]])
    else:
        J[np.abs(clause[0]),np.abs(clause[1])] += (0.25 if clause[0]*clause[1]>0 else -0.25)

### Embedding

In [68]:
embedding = {q: [q] for q in qubits}

### Solving in D'Wave QPU

In [69]:
num_reads=100
anneal_time=100.0
secure_q = input("Do you really want to use the QPU? (y/n)")
if secure_q == 'y' or secure_q == 'Y':
    sampler = FixedEmbeddingComposite(DWaveSampler(token='Your Token'), embedding=embedding)
    sample_set = sampler.sample_ising(h, J, num_reads=num_reads, annealing_time=anneal_time, return_embedding=True)
    print("Using DWaveSampler()")
    print(sample_set)

Do you really want to use the QPU? (y/n) y


Using DWaveSampler()
   300 301 302 303 304 305 306 307 308 309 310 311 ... 4936 energy num_oc. ...
0   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1 ...   +1  -38.5       1 ...
1   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1 ...   +1  -38.5       1 ...
2   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1 ...   +1  -38.5       1 ...
3   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1 ...   +1  -38.5       1 ...
4   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1 ...   +1  -38.5       1 ...
5   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1 ...   +1  -38.5       1 ...
6   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1 ...   +1  -38.5       1 ...
7   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1 ...   +1  -38.5       1 ...
8   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1 ...   +1  -38.5       1 ...
9   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1 ...   +1  -38.5       1 ...
10  -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1 ...   +1  -38.5       1 ...
11  +1  -1  +1  +1  -1  +1  +1 

In [70]:
qpu_access_time = sample_set.info['timing']['qpu_access_time']
#Tiempo usado por la QPU para resolver el problema de forma activa
qpu_sampling_time = sample_set.info['timing']['qpu_sampling_time']
print("Using DWave QPU... QPU access time:", qpu_access_time*10**(-6), "s.", " ; QPU sampling time:", qpu_sampling_time*10**(-6), "s.")

Using DWave QPU... QPU access time: 0.044994769999999996 s.  ; QPU sampling time: 0.029231999999999998 s.


In [71]:
dwave.inspector.show(sample_set)

Serving Inspector on http://127.0.0.1:18000/?problemId=f830ac0c-687b-416c-88c1-1c8cb0a14ecc

'http://127.0.0.1:18000/?problemId=f830ac0c-687b-416c-88c1-1c8cb0a14ecc'

#### Post-processing D'Wave

In [72]:
solver_greedy = SteepestDescentSolver()
sample_set_pp = solver_greedy.sample_ising(h, J, initial_states=sample_set)
print(sample_set_pp)

   300 301 302 303 304 305 306 307 308 309 310 311 ... 4936 energy num_oc. ...
0   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1 ...   +1  -38.5       1 ...
1   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1 ...   +1  -38.5       1 ...
2   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1 ...   +1  -38.5       1 ...
3   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1 ...   +1  -38.5       1 ...
4   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1 ...   +1  -38.5       1 ...
5   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1 ...   +1  -38.5       1 ...
6   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1 ...   +1  -38.5       1 ...
7   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1 ...   +1  -38.5       1 ...
8   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1 ...   +1  -38.5       1 ...
9   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1 ...   +1  -38.5       1 ...
10  -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1 ...   +1  -38.5       1 ...
11  +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1 .

### Simulated Annealing

In [73]:
ising_model = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.Vartype.SPIN)
sampler = neal.SimulatedAnnealingSampler()
start_time = time.time()
response = sampler.sample(ising_model, num_reads=100)
end_time = time.time()

print("Using SimulatedAnnlearingSampler()... Time:", round(end_time-start_time, 2), "s.")
print(response)

Using SimulatedAnnlearingSampler()... Time: 0.24 s.
   300 301 302 303 304 305 306 307 308 309 310 311 315 ... 4936 energy num_oc.
0   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  +1  +1  -1 ...   -1  -38.5       1
1   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1  -1 ...   +1  -38.5       1
2   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  +1  +1  +1 ...   -1  -38.5       1
3   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  +1  +1  +1 ...   +1  -38.5       1
4   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  +1  +1  +1 ...   +1  -38.5       1
5   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1  -1 ...   +1  -38.5       1
6   -1  -1  +1  +1  -1  +1  +1  +1  -1  -1  +1  +1  -1 ...   +1  -38.5       1
7   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1  -1 ...   +1  -38.5       1
8   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  +1  +1  -1 ...   +1  -38.5       1
10  -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  +1  +1  -1 ...   +1  -38.5       1
11  -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1  -1 ...   +1  -38.5       1


#### Post-processing Simulated Annealing

In [74]:
solver_greedy = SteepestDescentSolver()
response_pp = solver_greedy.sample_ising(h, J, initial_states=response)
print(response_pp)

   300 301 302 303 304 305 306 307 308 309 310 311 ... 4936 energy num_oc. ...
0   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  +1  +1 ...   -1  -38.5       1 ...
1   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1 ...   +1  -38.5       1 ...
2   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  +1  +1 ...   -1  -38.5       1 ...
3   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  +1  +1 ...   +1  -38.5       1 ...
4   +1  -1  +1  +1  -1  +1  +1  -1  +1  -1  +1  +1 ...   +1  -38.5       1 ...
5   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1 ...   +1  -38.5       1 ...
6   -1  -1  +1  +1  -1  +1  +1  +1  -1  -1  +1  +1 ...   +1  -38.5       1 ...
7   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  +1 ...   +1  -38.5       1 ...
8   -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  +1  +1 ...   +1  -38.5       1 ...
10  -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  +1  +1 ...   +1  -38.5       1 ...
11  -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  -1  -1 ...   +1  -38.5       1 ...
12  -1  -1  +1  +1  -1  +1  +1  -1  +1  -1  +1  +1 .

### Exact Solver (WMaxCDCL)

#### Rewrite a new sat file with the good format for Exact Solver

In [81]:
def normalize_wcnf(filename):
    i = 1
    with open(filename,"r") as f:
        with open(filename[:-5]+".norm.wcnf","w") as f_out:
            variables = {}
            all_variables={}
            for l in f:
                line = "1 "
                for var in l.split()[1:-1]:
                    all_variables[var]=True
                    negated = var[0]=='-'
                    if negated:
                        var = var[1:]
                    if var not in variables:
                        variables[var] = i
                        i += 1
                    line+=("-" if negated else "")
                    line+=str(variables[var])
                    line+=" "
                line+="0\n"   
                f_out.write(line)
    print(len(all_variables.keys()))

In [82]:
print("El numero total de variables (siendo negadas como nuevas variables) es:")
normalize_wcnf(file_name)

El numero total de variables (siendo negadas como nuevas variables) es:
166


#### Solving with UWRMaxSAT

In [83]:
!./../WMaxCDCL/bin/uwrmaxsat SAT_instance.norm.wcnf

c Using COMiniSatPS SAT solver by Chanseok Oh (2016)
c Parsing MaxSAT file...
c ============================[  Problem Statistics ]============================
c |  Number of variables:           309                                         |
c |  Number of clauses:             154                                         |
c ===============================================================================
c Using SCIP solver, version 8.0.0, https://www.scipopt.org
c Starting SCIP solver in a separate thread (with time-limit = 0s) ...
o 0
c Optimal solution: 0
c _______________________________________________________________________________
c 
c restarts               : 1
c conflicts              : 0              (-nan /sec)
c decisions              : 1              (0.00 % random) (inf /sec)
c propagations           : 308            (inf /sec)
c conflict literals      : 0              (-nan % deleted)
c =======================[ UWrMaxSat resources usage ]===========================
c Memo

### Aproximate solver (NuWLS-c)

In [84]:
%cd ../NuWLS-c-main/bin/
!./starexec_run_default-runsolver ../../nb/SAT_instance.norm.wcnf
%cd ../../nb/

/home/budyblack/QuantumSAT/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/budyblack/QuantumSAT/NuWLS-c-main/bin
c [MSE2022] -- begin -- [MSE2022]
0.00/0.03	c
0.00/0.03	c NuWLS based on TT-Open-WBO-Inc:	 an Anytime MaxSAT Solver --- based on Glucose4.1 (core version)
0.00/0.03	c Version:	 MaxSAT Evaluation 2021
0.00/0.03	c Author of NuWLS:	 Yi Chu, Xiang He
0.00/0.03	c Author of TT-Open-WBO-Inc:	 Alexander Nadel
0.00/0.03	c Authors of the baseline solver Open-WBO-Inc:	 Saurabh Joshi, Prateek Kumar, Ruben Martins, Sukrut Rao
0.00/0.03	c WARNING: for repeatability, setting FPU to use double precision
0.00/0.03	c |                                                                                                       |
0.00/0.03	c ========================================[ Problem Statistics ]===========================================
0.00/0.03	c |                                                                                                       |
0.00/0.03	c |  Problem Format:             MaxSAT                                                            

### Comparison

In [86]:
def count_unsatisfied_clauses(assignment, clauses):
    count = 0
    for clause in clauses:
        for literal in clause:
            if (literal > 0 and assignment[literal] == 1) or (literal < 0 and assignment[-literal] == -1):
                count += 1
                break
    return str(len(clauses)-count)

In [87]:
unsat = count_unsatisfied_clauses(sample_set.first.sample, clauses)
unsat_sim = count_unsatisfied_clauses(response.first.sample, clauses)

print("Unsatisfied clauses for D'Wave best solution: ", unsat, "  QPU access time:", qpu_access_time*10**(-6), "s.")
print("For Simulated Annealing: ", unsat_sim, "  Time needed:", round(end_time-start_time, 2), "s.")

data = {'Solver': ['DWave QPU', 'Simmulated Annealing', 'Exact solver', 'Approx. solver'],
        'Best Energy': [sample_set.first.energy, response.first.energy, '-', '-'],
        'Post_proces.': [sample_set_pp.first.energy, response_pp.first.energy, '-', '-'],
        'Unsatisfied clauses': [unsat, unsat_sim, "-", "-"],
        'Time (no pp) (s)': [qpu_access_time*10**(-6), round(end_time-start_time, 2), "-", "-"]}

# Creating a DataFrame
df = pd.DataFrame(data)

# Displaying the DataFrame
print(df)

Unsatisfied clauses for D'Wave best solution:  0   QPU access time: 0.044994769999999996 s.
For Simulated Annealing:  0   Time needed: 0.24 s.
                 Solver Best Energy Post_proces. Unsatisfied clauses  \
0             DWave QPU       -38.5        -38.5                   0   
1  Simmulated Annealing       -38.5        -38.5                   0   
2          Exact solver           -            -                   -   
3        Approx. solver           -            -                   -   

  Time (no pp) (s)  
0         0.044995  
1             0.24  
2                -  
3                -  
